# Accessing your data in DataFrames

*Note: There is also a longer, more complete version of this information in the notebook [FullAccessingDataFrames](FullAccessingDataFrames.ipynb)*

One big problem with Pandas is that, sometimes for historical reasons, there are multiple ways of doing the same thing. **The syntax for accessing DataFrames has always been one of the most confusing aspects for me!**

Let's make a toy DataFrame to see some of the features

---

*To preserve the mystery, select from the notebook menus*

`Edit -> Clear All Outputs`

---

In [1]:
import pandas as pd
import polars as pl

In [2]:
data_dict = {'letters':['A','B','c','D','eee'], 
             'hundreds':[100,200,300,400,500], 
             'tens':[10.0,20.0,30.0,40.0,50.0],
             'boolean':[True,False,True,True,False]}

In [3]:
df = pl.DataFrame(data_dict)
df

letters,hundreds,tens,boolean
str,i64,f64,bool
"""A""",100,10.0,true
"""B""",200,20.0,false
"""c""",300,30.0,true
"""D""",400,40.0,true
"""eee""",500,50.0,false


## DataFrame Attributes

### Each column has a data "type"

- **object** is how Pandas refers to strings of text
- **int64** is a 64-bit integer (whole number). The number of bits is just the amount of internal storage used for that number. *For integers it limits how big the number can be.* **Note that the default int64 can not store NaN/Null values!** See the [integer NA documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/integer_na.html) for more information.
- **float64** is a "floating point" number (number with decimal places). *For floats the number of bits limits the precision of the number.*
- **bool** is a booleal value, which is just True/False

In [50]:
df.dtypes

[String, Int64, Float64, Boolean]

In [4]:
df.schema

Schema([('letters', String),
        ('hundreds', Int64),
        ('tens', Float64),
        ('boolean', Boolean)])

### DataFrame index

Notice the column of sequential integers off to the left-hand side of the DataFrame output. That is the DataFrame's **index**. 

- **The index contains the names of the rows**
- Because we didn't explicitly specify an index column, Pandas created one for us

### DataFrame columns

There is a separate index of column names

In [5]:
df.columns

['letters', 'hundreds', 'tens', 'boolean']

---

## `df[]` with a name for column indexing/selecting

The most common, and concise, way of selecting a column out of a DataFrame is just using square brackets with the column name inside – similar to how you access a dictionary value using it's key.

### Single name gives single column

This returns a Series

In [6]:
df.select('hundreds')

hundreds
i64
100
200
300
400
500


### List of column names inside the square brackets gives multiple columns

You can select multiple columns by putting a *list of column names* inside the square brackets. This returns a DataFrame

In [8]:
df.select('tens','hundreds')

tens,hundreds
f64,i64
10.0,100
20.0,200
30.0,300
40.0,400
50.0,500


In [11]:
df.select(pl.col('tens','hundreds'))

tens,hundreds
f64,i64
10.0,100
20.0,200
30.0,300
40.0,400
50.0,500


In [10]:
df.select(pl.col(pl.Int64))

hundreds
i64
100
200
300
400
500


### Polars Selectors

https://docs.pola.rs/api/python/stable/reference/selectors.html

In [12]:
import polars.selectors as cs

In [13]:
df.select(cs.numeric())

hundreds,tens
i64,f64
100,10.0
200,20.0
300,30.0
400,40.0
500,50.0


In [14]:
df.select(cs.string())

letters
str
"""A"""
"""B"""
"""c"""
"""D"""
"""eee"""


In [15]:
df.select(cs.boolean())

boolean
bool
true
false
true
true
false


In [25]:
df.select(cs.contains('en'))

tens
f64
10.0
20.0
30.0
40.0
50.0


In [17]:
df.select(pl.all())

letters,hundreds,tens,boolean
str,i64,f64,bool
"""A""",100,10.0,true
"""B""",200,20.0,false
"""c""",300,30.0,true
"""D""",400,40.0,true
"""eee""",500,50.0,false


In [30]:
df.select(pl.all().exclude('^boo.*$'))

letters,hundreds,tens
str,i64,f64
"""A""",100,10.0
"""B""",200,20.0
"""c""",300,30.0
"""D""",400,40.0
"""eee""",500,50.0


In [31]:
df.select(cs.exclude('^boo.*$',pl.Int64))

letters,tens
str,f64
"""A""",10.0
"""B""",20.0
"""c""",30.0
"""D""",40.0
"""eee""",50.0


---

## `df[]` with boolean Series returns multiple rows, all columns

### Boolean Series

**It's very common to want all the rows from your DataFrame which pass a certain test**, or set of criteria. You can write the test itself in a very straightforward way, returning a `Series` of True/False boolean values.

In [16]:
pl.col('tens') < 35

<Expr ['[(col("tens")) < (dyn int: 35)…'] at 0x127D50F50>

In [32]:
df.select(pl.col('tens') < 35)

tens
bool
true
true
true
false
false


In [36]:
df.select((pl.col('tens') < 35).alias('tens_gt_35'))

tens_gt_35
bool
true
true
true
false
false


### Rows are returned where boolean Series == True

**Things are getting screwy! If we use a single bracket with a boolean Series inside, we get back rows instead of columns!**

*This can be confusing, but it's convenient enough that you'll see it quite often. Note that you can easily use the `df.loc[,:]` notation below to do the same thing, which is somewhat more clear and readable.*


In [33]:
df.filter(pl.col('tens') < 35)

letters,hundreds,tens,boolean
str,i64,f64,bool
"""A""",100,10.0,true
"""B""",200,20.0,false
"""c""",300,30.0,true


In [34]:
df.filter(pl.col('boolean'))

letters,hundreds,tens,boolean
str,i64,f64,bool
"""A""",100,10.0,true
"""c""",300,30.0,true
"""D""",400,40.0,true


### More complicated conditionals

You can combine multiple conditions if you put them in parentheses and put a logical operator between.

- `&` = "and"
- `|` = "or"  *(It's called a "pipe" character, and it's `shift-\` above the Enter/Return key)*

In [37]:
df.filter((pl.col('tens') < 35) & (pl.col('hundreds') > 200))

letters,hundreds,tens,boolean
str,i64,f64,bool
"""c""",300,30.0,true


---

*To try the exercise below, if you haven't been executing the cells above, select this cell and from the Jupyter menus choose*

`Run -> Run All Above Selected Cell`

### **EXERCISE**

**Return all rows in `df` where the "hundreds" column value is greater than or equal to 400**

*Note: Type instead of using copy/paste for better retention*

---

## Row Index doesn't have to be integers!

The Index values aren't "row numbers". Instead, **the Index values are the names of the rows, so you can use things like strings or dates for the Index**. Let's start by making a new column.

*Note that if we creating a new column from a list we need to make sure we don't make any mistakes in the order of the DataFrame rows compared to the order in our list! A more reliable solution would be to create a Series with explicit Index values and assign to the new column from that. If you initialize a Series with a Dictionary, the key values will be used as the row Index. Try reordering the key value pairs and verify that the row assignments stay the same!*

*Better:* `df['spelled_out'] = pd.Series({0:'One', 1:'Two', 2:'Three', 3:'Four', 4:'Five'})`

Here we'll just do it the simpler way...

### We can set one of the columns as the Index

### Every column's Series has this same Index

---

## `df.loc[row,col]` for label-based, multi-axis indexing

**This is the best access/selection method!** It lets you select first along rows, and then along columns, in both directions simultaneoulsy using row and column "labels", which are the row index and column names.

In [38]:
df.item(0,'letters')

'A'

### Colon `:` for selecting whole or slices

A notation that comes from accessing lists in Python is the "slice" operator, which is specified with a colon between two values. 

- **The colon by itself denotes all Index entries in a row or column.**

So, here we return all rows and a single column.

### Slice end-points are included in Pandas selections

Unlike Python lists, in Pandas the slice includes both end points!


---

*To try the exercise below, if you haven't been executing the cells above, select this cell and from the Jupyter menus choose*

`Run -> Run All Above Selected Cell`

### **EXERCISE**

**Do you know how to use the slice notation to specify just the rows up to and including "Three"?**

*Note: Type instead of using copy/paste for better retention*

---

### Lists for combinations

Lists of values work the same as with the `df[]` notation. *Note that again, order matters for what is returned!*

### Single rows are a Series, too

- Remember, any 1D result, row or column, will be a Series in Pandas
- When you're returning a row, the Index will be the column names

In [42]:
df.row(0)

('A', 100, 10.0, True)

### Boolean Series can be used for selecting True rows or columns

But notice with this syntax you can tell easily whether you're using the boolean Series to select rows or columns, unlike with the `df[]` syntax.

---

*To try the exercise below, if you haven't been executing the cells above, select this cell and from the Jupyter menus choose*

`Run -> Run All Above Selected Cell`

### **EXERCISE**

**Using the `df2.loc[]` notation, return all columns for all rows in which the "boolean" column is True**

*Note: Type instead of using copy/paste for better retention*

---

## `df.query()` for selecting rows like in SQL "where" statement

**The idea is to return rows where column content meets certain criteria, like when you use a boolean series inside of a `df[]` statement**

This is not a syntax I'm very experienced with, but people coming from the SQL (Structured Query Language) relational database query world might feel more comfortable with this form. It does sometimes have the advantage of shorter, more readable conditional expressions.

Some bulleted text below is from the Pandas [DataFrame.query documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html):

### Format your query as a string in quotes

- You can refer to column names (that don't have spaces) directly in the expression

In [48]:
df.sql("""
    SELECT * FROM self
    WHERE tens < 35
    """)

letters,hundreds,tens,boolean
str,i64,f64,bool
"""A""",100,10.0,true
"""B""",200,20.0,false
"""c""",300,30.0,true


### Query can refer to variables in the environment

- You can refer to variables in the environment by prefixing them with an ‘@’ character

### Need backticks around column names with spaces

- You can refer to column names that are not valid Python variable names by surrounding them in backticks. Thus, column names containing spaces or punctuations (besides underscores) or starting with digits must be surrounded by backticks. (For example, a column named “Area (cm^2)” would be referenced as \`Area (cm^2)\`). 
- Strings in the conditional need the opposite type of quotes than you've used to surround the expression itself


*Note that this returns a DataFrame even for a single row, which is actually the same behavior as*

```
df2[df2['more letters'] == 'DD']
```

### Complex conditionals use "and" / "or"

- Logical expressions can either use "and" or "or", or "&" or "|" for more complex conditionals.

**This is where the queries are slightly more readable than the other form**

```
df2[(df2['tens'] < 35) & (df2['boolean'] == True)]
```

---

## I've skipped the integer selection methods

Indexing by integers using the `df.iloc[row index, column index]` method is good to know about, but frankly, I hardly ever find myself using that method – I prefer to use labels. 

*I never use integers inside the `df[]` notation – it is just a historical leftover that should be avoided!*

See the [FullAccessingDataFrames notebook](FullAccessingDataFrames.ipynb) to learn more.

---

## SettingWithCopyWarning

If you select a subset of your DataFrame and assign to a new variable as a way to keep just a couple columns of your DataFrame, you will run into a **SettingWithCopyWarning** if you try to change that new DataFrame. 

*Because of some Pandas inner-workings, when you slice or index into a DataFrame it's not actually clear whether it will create a "view" into the original DataFrame or a copy of the data!* 

In [57]:
df_nums = df.select(pl.col('hundreds','tens'))
df_nums

hundreds,tens
i64,f64
100,10.0
200,20.0
300,30.0
400,40.0
500,50.0


### ~~It does the operation but gives you a confusing warning~~

You don't want these warnings all over the place in your notebook or you'll miss really important errors and warnings, and there's a chance you might be doing something wrong, anyway!

In [58]:
df_nums.with_columns(sums = pl.col('hundreds') + pl.col('tens'))


hundreds,tens,sums
i64,f64,f64
100,10.0,110.0
200,20.0,220.0
300,30.0,330.0
400,40.0,440.0
500,50.0,550.0


In [59]:
df_nums.with_columns((pl.col('hundreds') + pl.col('tens')).alias('sums'))

hundreds,tens,sums
i64,f64,f64
100,10.0,110.0
200,20.0,220.0
300,30.0,330.0
400,40.0,440.0
500,50.0,550.0


### Create an explicit `.copy()`

**If you are assigning a subset of a DataFrame to a new variable**, with the intention of creating a copy that you'll work on independently of the original, **create an explicit copy by chaining the `.copy()` method to the end!**

Additional learning resources:
- [The best explanation of the SettingWithCopyWarning I've seen](https://www.dataquest.io/blog/settingwithcopywarning/)
- [A complicated explanation in the Pandas documentation](https://pandas.pydata.org/pandas-docs/version/0.22/indexing.html#indexing-view-versus-copy)


---

## `df.loc[row,col]` for setting values

So far we've been accessing values in DataFrames to read/get what's already there, but **the same methods can be used for setting values on DataFrame subsets**!

---

## Selection method you'll see, but don't really need!

A very good article on more advanced Pandas features where there are multiple ways of doing similar things, and [Ted Petrou's](https://medium.com/@petrou.theodore) 
opinions on which to use, is 
[Minimally Sufficient Pandas](https://medium.com/dunder-data/minimally-sufficient-pandas-a8e67f2a2428). 

*These pieces are taken from that article, but there's a lot more great content there that we don't have time to cover.*

### Selecting a single column with the "dot" notation

A very common alternative to selecting a single column with the `df['name']` bracket notation you'll see all the time, is what's called the "dot" notation, where you follow the dataframe name by a dot and the column name, `df.name`

### Issues with the "dot" notation

There are three issues with using dot notation. It doesn’t work in the following situations:

- When there are spaces in the column name
- When the column name is the same as a DataFrame method
- When the name of a column you want to access is stored in a variable

#### Spaces in the columns name

#### Column name is the same as a DataFrame method

`sum` can be the name of one of our columns, and it's easy to access those values with the "quoted name in square brackets" notation:

**But if we try to access that column's values with the "dot" notation, there is a problem.** The output here will seem confusing, but it's basically saying that `.sum` is a method (built-in function) of the DataFrame.

#### The column name is stored in a variable

It's not uncommon to want to access a column who's name string has been stored in a variable

**But, again, that doesn't work with the "dot" notation**

### Lots of Pandas is written with the dot notation. Why?

Many tutorials make use of the dot notation to select a single column of data. Why is this done when the brackets seem to be clearly superior? 

- It might be because **the official documentation contains plenty of examples that use it**
- It also ***uses three fewer characters which entices the very laziest amongst us***

---